In [2]:
import pandas as pd

# Load the Excel file again and preview the first few rows of each sheet
xls = pd.ExcelFile(r'C:\Users\User\Desktop\PROJEKTY_VS\ICY24-CapeTown\transcriptomics20240424\PKW-Supplement-Global_transcriptome_profiling.xlsx')
sample_data = {}

# Loop through each sheet and collect the first few rows
for sheet_name in xls.sheet_names:
    df = pd.read_excel(xls, sheet_name=sheet_name)
    sample_data[sheet_name] = df.head()

sample_data

{'SoA_all_genes':             Gene  BaseMean  Estimated_FoldChange  log2_Estimated_FoldChange  \
 0  YALI0_B00374g  19113.96                  0.04                      -4.58   
 1  YALI0_F15411g   5344.51                 19.92                       4.32   
 2  YALI0_F19030g   7841.18                  0.12                      -3.09   
 3  YALI0_E05819g   5053.00                  0.21                      -2.27   
 4  YALI0_B04202g   1911.87                  0.04                      -4.78   
 
    p_value_not_adjusted  FDR_adjusted_p_value  Standard_error_estimate  \
 0          0.000000e+00          0.000000e+00                     0.09   
 1          0.000000e+00          0.000000e+00                     0.11   
 2         1.692696e-211         3.502752e-208                     0.10   
 3         9.301683e-211         1.443621e-207                     0.07   
 4         1.805869e-208         2.242167e-205                     0.16   
 
    Sample_scSoA_1_raw_counts  Sample_scSoA_2_raw

In [3]:
pd.options.display.max_rows = 1000
pd.options.display.max_columns = 30

# Function to extract the unique part of the gene ID from the "Gene" column
def extract_unique_gene_id(gene_id):
    if isinstance(gene_id, str):
        try:
            return gene_id.split('_')[1][:-1]
        except IndexError:
            return gene_id
    return gene_id

# Read the gene list from the text file
gene_list_path = r'C:\Users\User\Desktop\PROJEKTY_VS\ICY24-CapeTown\transcriptomics20240424\TFlib_1_lista_genów.txt'
with open(gene_list_path, 'r') as file:
    gene_list = file.read().splitlines()

# Load the Excel file
excel_path = r'C:\Users\User\Desktop\PROJEKTY_VS\ICY24-CapeTown\transcriptomics20240424\PKW-Supplement-Global_transcriptome_profiling.xlsx'
xls = pd.ExcelFile(excel_path)
filtered_dataframes = {}

# Loop through each sheet in the Excel file and create a dataframe for each
for sheet_name in xls.sheet_names:
    df = pd.read_excel(xls, sheet_name=sheet_name)
    df['unique_gene_id'] = df['Gene'].apply(extract_unique_gene_id)
    filtered_dataframes[sheet_name] = df[df['unique_gene_id'].isin(gene_list)].drop(columns=['unique_gene_id'])

# Show the first few rows of each filtered dataframe to verify
filtered_dataframes_preview = {sheet: df.head() for sheet, df in filtered_dataframes.items()}
print(filtered_dataframes_preview)

# Save filtered dataframes to new Excel file
output_path = r'C:\Users\User\Desktop\PROJEKTY_VS\ICY24-CapeTown\transcriptomics20240424\Filtered_Gene_Data_PKW.xlsx'
with pd.ExcelWriter(output_path) as writer:
    for sheet_name, df in filtered_dataframes.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)

print(f"Filtered data has been saved to {output_path}")


{'SoA_all_genes':               Gene  BaseMean  Estimated_FoldChange  log2_Estimated_FoldChange  \
14   YALI0_D20482g    706.80                  0.05                      -4.20   
251  YALI0_E20251g   3676.07                  0.43                      -1.22   
263  YALI0_C03564g    583.81                  2.79                       1.48   
280  YALI0_C18645g   2452.41                  1.78                       0.84   
308  YALI0_D01573g  14421.85                  1.48                       0.57   

     p_value_not_adjusted  FDR_adjusted_p_value  Standard_error_estimate  \
14          8.732142e-117         3.613943e-114                     0.18   
251          5.975680e-19          1.472104e-17                     0.14   
263          2.959798e-18          6.960010e-17                     0.17   
280          1.315106e-17          2.905401e-16                     0.10   
308          1.370792e-16          2.754005e-15                     0.07   

     Sample_scSoA_1_raw_counts  Sample

In [4]:
# Initialize an empty dataframe to store the merged results
merged_df = pd.DataFrame()

# Loop through each filtered dataframe and extract necessary columns
for sheet_name, df in filtered_dataframes.items():
    # Extract Gene and log2_Estimated_FoldChange columns
    extracted_df = df[['Gene', 'log2_Estimated_FoldChange']].copy()
    # Rename log2_Estimated_FoldChange to include the sheet name
    extracted_df.rename(columns={'log2_Estimated_FoldChange': f'log2_Estimated_FoldChange_{sheet_name}'}, inplace=True)
    
    # Merge with the main dataframe
    if merged_df.empty:
        merged_df = extracted_df
    else:
        merged_df = pd.merge(merged_df, extracted_df, on='Gene', how='outer')

# Save the merged dataframe to a new Excel file
output_path = r'C:\Users\User\Desktop\PROJEKTY_VS\ICY24-CapeTown\transcriptomics20240424\Merged_Gene_Data_PKW.xlsx'
merged_df.to_excel(output_path, index=False)

print(f"Merged data has been saved to {output_path}")

Merged data has been saved to C:\Users\User\Desktop\PROJEKTY_VS\ICY24-CapeTown\transcriptomics20240424\Merged_Gene_Data_PKW.xlsx


In [5]:
# Load the new Excel file
new_excel_path = r'C:\Users\User\Desktop\PROJEKTY_VS\ICY24-CapeTown\transcriptomics20240424\PKW-Supplement-Molecular_background_HAC1.xlsx'
new_xls = pd.ExcelFile(new_excel_path)

# Define the sheets to process
sheets_to_process = ['UP_sort_FC', 'DOWN_sort FC']
filtered_dataframes = []

# Loop through each relevant sheet and filter the data
for sheet_name in sheets_to_process:
    df = pd.read_excel(new_xls, sheet_name=sheet_name)
    df['unique_gene_id'] = df['MAPPED IDs'].apply(extract_unique_gene_id)
    filtered_df = df[df['unique_gene_id'].isin(gene_list)].drop(columns=['unique_gene_id'])
    filtered_dataframes.append(filtered_df)

# Merge the filtered dataframes
merged_df = pd.concat(filtered_dataframes, ignore_index=True)

# Save the merged dataframe to a new Excel file
output_path = r'C:\Users\User\Desktop\PROJEKTY_VS\ICY24-CapeTown\transcriptomics20240424\Filtered_Gene_Data_Hac.xlsx'
merged_df.to_excel(output_path, index=False)

print(f"Filtered and merged data has been saved to {output_path}")

Filtered and merged data has been saved to C:\Users\User\Desktop\PROJEKTY_VS\ICY24-CapeTown\transcriptomics20240424\Filtered_Gene_Data_Hac.xlsx


In [6]:
import pandas as pd

# Define paths
previous_merged_data_path = r'C:\Users\User\Desktop\PROJEKTY_VS\ICY24-CapeTown\transcriptomics20240424\Merged_Gene_Data_PKW.xlsx'
new_filtered_merged_data_path = r'C:\Users\User\Desktop\PROJEKTY_VS\ICY24-CapeTown\transcriptomics20240424\Filtered_Gene_Data_Hac.xlsx'
output_combined_path = r'C:\Users\User\Desktop\PROJEKTY_VS\ICY24-CapeTown\transcriptomics20240424\Combined_Gene_Data_PKW.xlsx'

# Load the previously merged data
previous_merged_df = pd.read_excel(previous_merged_data_path)

# Load the new filtered and merged data
new_filtered_merged_df = pd.read_excel(new_filtered_merged_data_path)

# Extract relevant columns from dataset
new_extracted_df = new_filtered_merged_df[['MAPPED IDs', 'log2_Estimated_FoldChange']]

# Rename columns to match for merging
new_extracted_df.rename(columns={'MAPPED IDs': 'Gene', 'log2_Estimated_FoldChange': 'log2_Estimated_FoldChange_HAC_scYFP'}, inplace=True)

# Merge datasets on 'Gene' column
combined_df = pd.merge(previous_merged_df, new_extracted_df, on='Gene', how='outer')

# Save the final combined dataframe to a new Excel file
combined_df.to_excel(output_combined_path, index=False)

print(f"Combined data has been saved to {output_combined_path}")


Combined data has been saved to C:\Users\User\Desktop\PROJEKTY_VS\ICY24-CapeTown\transcriptomics20240424\Combined_Gene_Data_PKW.xlsx


C:\Users\User\AppData\Local\Temp\ipykernel_26364\2854581638.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_extracted_df.rename(columns={'MAPPED IDs': 'Gene', 'log2_Estimated_FoldChange': 'log2_Estimated_FoldChange_HAC_scYFP'}, inplace=True)


In [7]:
combined_df.fillna(0, inplace=True)
combined_df.rename(columns={"log2_Estimated_FoldChange_SoA_all_genes": "scSoA", "log2_Estimated_FoldChange_scYFP_all_genes": "scYFP",
                            "log2_Estimated_FoldChange_inYFP_all_genes": "inYFP", "log2_Estimated_FoldChange_TlG_all_genes" : "scTlG",
                            "log2_Estimated_FoldChange_HAC_scYFP": "HAC_scYFP"}, inplace=True)

print(combined_df)

             Gene  scSoA  scYFP  inYFP  scTlG  HAC_scYFP
0   YALI0_D20482g  -4.20  -0.67   0.00   0.00       0.00
1   YALI0_E20251g  -1.22  -0.70   0.00   0.22       0.00
2   YALI0_C03564g   1.48   1.08   0.00   0.72       1.02
3   YALI0_C18645g   0.84   0.00   0.00  -0.28       0.00
4   YALI0_D01573g   0.57   0.36   0.00   0.00       0.00
5   YALI0_E16973g  -0.62  -0.49   0.00   0.00       0.00
6   YALI0_F11979g  -0.62  -0.34   0.00   0.00       0.00
7   YALI0_D14872g  -2.15  -0.64   1.30   1.40       0.00
8   YALI0_D01353g  -2.73  -2.15  -2.44  -2.79      -2.34
9   YALI0_E31383g  -0.84  -0.45   0.00  -0.26      -0.49
10  YALI0_E24277g   0.43   0.00   0.00  -0.24      -0.36
11  YALI0_D24167g   0.63   0.45   0.00   0.00       0.00
12  YALI0_B19602g  -2.41  -1.38   0.00   0.00       0.00
13  YALI0_D18678g   0.50   0.41   0.00   0.20       0.00
14  YALI0_B00660g  -0.98  -0.62   0.50   0.61       0.00
15  YALI0_B21582g  -1.15  -0.84   0.00   0.18       0.00
16  YALI0_F11011g   0.51   0.24

In [17]:
name_table = pd.read_excel('Table_TF-Gene-Name.xlsx')
name_table.fillna('-', inplace=True)
name_table.drop([0], axis=0, inplace=True)
print(name_table.head())

  TF number           Gene Assigned name
1     TF001  YALI0_A10637g             -
2     TF002  YALI0_A12925g             -
3     TF003  YALI0_B00660g             -
4     TF004  YALI0_B06853g          PUT3
5     TF005  YALI0_B09713g          PPR1


In [18]:
final_transcriptomics = pd.merge(combined_df, name_table, on='Gene', how='outer')
print(final_transcriptomics.head(30))

             Gene  scSoA  scYFP  inYFP  scTlG  HAC_scYFP TF number  \
0   YALI0_D20482g  -4.20  -0.67   0.00   0.00       0.00     TF037   
1   YALI0_E20251g  -1.22  -0.70   0.00   0.22       0.00     TF136   
2   YALI0_C03564g   1.48   1.08   0.00   0.72       1.02     TF013   
3   YALI0_C18645g   0.84   0.00   0.00  -0.28       0.00     TF121   
4   YALI0_D01573g   0.57   0.36   0.00   0.00       0.00     TF115   
5   YALI0_E16973g  -0.62  -0.49   0.00   0.00       0.00     TF104   
6   YALI0_F11979g  -0.62  -0.34   0.00   0.00       0.00     TF106   
7   YALI0_D14872g  -2.15  -0.64   1.30   1.40       0.00     TF034   
8   YALI0_D01353g  -2.73  -2.15  -2.44  -2.79      -2.34     TF024   
9   YALI0_E31383g  -0.84  -0.45   0.00  -0.26      -0.49     TF051   
10  YALI0_E24277g   0.43   0.00   0.00  -0.24      -0.36     TF049   
11  YALI0_D24167g   0.63   0.45   0.00   0.00       0.00     TF110   
12  YALI0_B19602g  -2.41  -1.38   0.00   0.00       0.00     TF010   
13  YALI0_D18678g   

In [21]:
final_transcriptomics.to_excel('final_transcriptomics.xlsx')

Zmiana nazwy tabeli na FINAL TRANSCRIPTOMICS

In [9]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import LinearSegmentedColormap


#heatmap = combined_df.drop(['Gene'], axis=1, inplace=True)

# Generate example data
data = np.random.randn(10, 10)

# Create a custom colormap with white around 0
colors = [(0, 'blue'), (0.5, 'white'), (1, 'red')]  # R -> G -> B
n_bins = 100  # Discretizes the interpolation into bins
cmap_name = 'coolwarm_white'
custom_cmap = LinearSegmentedColormap.from_list(cmap_name, colors, N=n_bins)

# Plot heatmap with the custom colormap
sns.heatmap(combined_df, cmap=custom_cmap, center=0)
plt.title('Transcriptome TFs')
plt.show()


ValueError: could not convert string to float: 'YALI0_D20482g'